# 📊 Dataset Maker by Hollowstrawberry

This is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb). Thank you!

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning inference.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **XL Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |

In [1]:
import os
from IPython import get_ipython
from IPython.display import display, Markdown

COLAB = True

if COLAB:
  from google.colab.output import clear as clear_output
else:
  from IPython.display import clear_output

#@title ## 🚩 Start Here

#@markdown ### 1️⃣ Setup
#@markdown This cell will load some requirements and create the necessary folders in your Google Drive. <p>
#@markdown Your project name can't contain spaces but it can contain a single / to make a subfolder in your dataset.
project_name = "ppzocketv2" #@param {type:"string"}
project_name = project_name.strip()
#@markdown The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same one. I like organizing by project.
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]

if not project_name or any(c in project_name for c in " .()\"'\\") or project_name.count("/") > 1:
  print("Please write a valid project_name.")
else:
  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  project_base = project_name if "/" not in project_name else project_name[:project_name.rfind("/")]
  project_subfolder = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]

  root_dir = "/content" if COLAB else "~/Loras"
  deps_dir = os.path.join(root_dir, "deps")

  if "/Loras" in folder_structure:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
    config_folder = os.path.join(main_dir, project_base)
    images_folder = os.path.join(main_dir, project_base, "dataset")
    if "/" in project_name:
      images_folder = os.path.join(images_folder, project_subfolder)
  else:
    main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
    config_folder = os.path.join(main_dir, "config", project_name)
    images_folder = os.path.join(main_dir, "datasets", project_name)

  for dir in [main_dir, deps_dir, images_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  print(f"✅ Project {project_name} is ready!")
  step1_installed_flag = True


📂 Connecting to Google Drive...
Mounted at /content/drive
✅ Project ppzocketv2 is ready!


In [2]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 4️⃣ Tag your images
#@markdown We will be using AI to automatically tag your images, specifically [Waifu Diffusion](https://huggingface.co/SmilingWolf/wd-v1-4-swinv2-tagger-v2) in the case of anime and [BLIP](https://huggingface.co/spaces/Salesforce/BLIP) in the case of photos.
#@markdown Giving tags/captions to your images allows for much better training. This process should take a couple minutes. <p>
method = "Photo captions" #@param ["Anime tags", "Photo captions"]
#@markdown **Anime:** The threshold is the minimum level of confidence the tagger must have in order to include a tag. Lower threshold = More tags. Recommended 0.35 to 0.5
tag_threshold = 0.35 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
blacklist_tags = "" #@param {type:"string"}
#@markdown **Photos:** The minimum and maximum length of tokens/words in each caption.
caption_min = 10 #@param {type:"number"}
caption_max = 75 #@param {type:"number"}

%env PYTHONPATH=/env/python
os.chdir(root_dir)
kohya = "/content/kohya-trainer"
if not os.path.exists(kohya):
  !git clone https://github.com/kohya-ss/sd-scripts {kohya}
  os.chdir(kohya)
  !git reset --hard 9a67e0df390033a89f17e70df5131393692c2a55
  os.chdir(root_dir)

if "tags" in method:
  if "step4a_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    !pip install accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 tensorflow transformers safetensors huggingface-hub torchvision albumentations jax==0.4.23 jaxlib==0.4.23
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      clear_output()
      step4a_installed_flag = True
    else:
      print("❌ Error installing dependencies, trying to continue anyway...")

  print("\n🚶‍♂️ Launching program...\n")

  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/tag_images_by_wd14_tagger.py \
    {images_folder} \
    --repo_id=SmilingWolf/wd-v1-4-swinv2-tagger-v2 \
    --model_dir={root_dir} \
    --thresh={tag_threshold} \
    --batch_size=8 \
    --caption_extension=.txt \
    --force_download

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    print("removing underscores and blacklist...")
    blacklisted_tags = [t.strip() for t in blacklist_tags.split(",")]
    from collections import Counter
    top_tags = Counter()
    for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(images_folder, txt), 'r') as f:
        tags = [t.strip() for t in f.read().split(",")]
        tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
        tags = [t for t in tags if t not in blacklisted_tags]
      top_tags.update(tags)
      with open(os.path.join(images_folder, txt), 'w') as f:
        f.write(", ".join(tags))

    %env PYTHONPATH=/env/python
    clear_output()
    print(f"📊 Tagging complete. Here are the top 50 tags in your dataset:")
    print("\n".join(f"{k} ({v})" for k, v in top_tags.most_common(50)))


else: # Photos
  if "step4b_installed_flag" not in globals():
    print("\n🏭 Installing dependencies...\n")
    !pip install timm==0.6.12 fairscale==0.4.13 transformers==4.26.0 requests==2.28.2 accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6 jax==0.4.23 jaxlib==0.4.23
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      clear_output()
      step4b_installed_flag = True
    else:
      print("❌ Error installing dependencies, trying to continue anyway...")

  print("\n🚶‍♂️ Launching program...\n")

  os.chdir(kohya)
  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/make_captions.py \
    {images_folder} \
    --beam_search \
    --max_data_loader_n_workers=2 \
    --batch_size=8 \
    --min_length={caption_min} \
    --max_length={caption_max} \
    --caption_extension=.txt

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    import random
    captions = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
    sample = []
    for txt in random.sample(captions, min(10, len(captions))):
      with open(os.path.join(images_folder, txt), 'r') as f:
        sample.append(f.read())

    os.chdir(root_dir)
    %env PYTHONPATH=/env/python
    clear_output()
    print(f"📊 Captioning complete. Here are {len(sample)} example captions from your dataset:")
    print("".join(sample))


📊 Captioning complete. Here are 10 example captions from your dataset:
a pair of wooden shoes with straps on them
a bottle of perfume sitting on a table
a pair of purple and black shoes hanging from a string
a jar of body butter next to a plant and rocks
a watch with a black face and orange hands
a smart watch sitting on top of a table next to a phone
a bottle of cbd oil and a jar of cbd cream
a bottle of beer sitting on a table
three different flavors of soda sitting on a pool
three orange suitcases sitting on a stone bench



In [3]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 5️⃣ Curate your tags
#@markdown Modify your dataset's tags. You can run this cell multiple times with different parameters. <p>

#@markdown Put an activation tag at the start of every text file. This is useful to make learning better and activate your Lora easier. Set `keep_tokens` to 1 when training.<p>
#@markdown Common tags that are removed such as hair color, etc. will be "absorbed" by your activation tag.
global_activation_tag = "ppzocketv2" #@param {type:"string"}
remove_tags = "" #@param {type:"string"}
#@markdown &nbsp;

#@markdown In this advanced section, you can search text files containing matching tags, and replace them with less/more/different tags. If you select the checkbox below, any extra tags will be put at the start of the file, letting you assign different activation tags to different parts of your dataset. Still, you may want a more advanced tool for this.
search_tags = "" #@param {type:"string"}
replace_with = "" #@param {type:"string"}
search_mode = "OR" #@param ["OR", "AND"]
new_becomes_activation_tag = False #@param {type:"boolean"}
#@markdown These may be useful sometimes. Will remove existing activation tags, be careful.
sort_alphabetically = False #@param {type:"boolean"}
remove_duplicates = False #@param {type:"boolean"}

def split_tags(tagstr):
  return [s.strip() for s in tagstr.split(",") if s.strip()]

activation_tag_list = split_tags(global_activation_tag)
remove_tags_list = split_tags(remove_tags)
search_tags_list = split_tags(search_tags)
replace_with_list = split_tags(replace_with)
replace_new_list = [t for t in replace_with_list if t not in search_tags_list]

replace_with_list = [t for t in replace_with_list if t not in replace_new_list]
replace_new_list.reverse()
activation_tag_list.reverse()

remove_count = 0
replace_count = 0

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:

  with open(os.path.join(images_folder, txt), 'r') as f:
    tags = [s.strip() for s in f.read().split(",")]

  if remove_duplicates:
    tags = list(set(tags))
  if sort_alphabetically:
    tags.sort()

  for rem in remove_tags_list:
    if rem in tags:
      remove_count += 1
      tags.remove(rem)

  if "AND" in search_mode and all(r in tags for r in search_tags_list) \
      or "OR" in search_mode and any(r in tags for r in search_tags_list):
    replace_count += 1
    for rem in search_tags_list:
      if rem in tags:
        tags.remove(rem)
    for add in replace_with_list:
      if add not in tags:
        tags.append(add)
    for new in replace_new_list:
      if new_becomes_activation_tag:
        if new in tags:
          tags.remove(new)
        tags.insert(0, new)
      else:
        if new not in tags:
          tags.append(new)

  for act in activation_tag_list:
    if act in tags:
      tags.remove(act)
    tags.insert(0, act)

  with open(os.path.join(images_folder, txt), 'w') as f:
    f.write(", ".join(tags))

if global_activation_tag:
  print(f"\n📎 Applied new activation tag(s): {', '.join(activation_tag_list)}")
if remove_tags:
  print(f"\n🚮 Removed {remove_count} tags.")
if search_tags:
  print(f"\n💫 Replaced in {replace_count} files.")
print("\n✅ Done! Check your updated tags in the Extras below.")



📎 Applied new activation tag(s): ppzocketv2

✅ Done! Check your updated tags in the Extras below.


In [ ]:
#@markdown ### 6️⃣ Ready
#@markdown You should be ready to [train your Lora](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb)!

from IPython.display import Markdown, display
display(Markdown(f"### 🦀 [Click here to open the Lora trainer](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb)"))


## *️⃣ Extras

In [4]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 📈 Analyze Tags
#@markdown Perhaps you need another look at your dataset.
show_top_tags = 50 #@param {type:"number"}

from collections import Counter
top_tags = Counter()

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
  with open(os.path.join(images_folder, txt), 'r') as f:
    top_tags.update([s.strip() for s in f.read().split(",")])

top_tags = Counter(top_tags)
print(f"📊 Top {show_top_tags} tags:")
for k, v in top_tags.most_common(show_top_tags):
  print(f"{k} ({v})")

📊 Top 50 tags:
ppzocketv2 (65)
a purse with a chain on a white surface (2)
a cup of coffee is being dropped into the air (1)
a camera with a lens on a black surface (1)
a black apple watch with a blue sky background (1)
a pair of red and white sneakers on a red background (1)
a pair of sunglasses sitting on top of a table (1)
a pair of white sneakers on a black background (1)
a table with a stool in front of it (1)
a car is shown in the dark on the road (1)
a plant is sitting on a table next to a cabinet (1)
a couch with a pillow and some lemons on the floor (1)
a pair of wooden shoes with straps on them (1)
a bottle of hemp oil sitting on a table (1)
a pair of sneakers with colorful laces on them (1)
a pair of shoes with wheels on a white platform (1)
a hand holding a pink flamingo purse (1)
a pair of black and white shoes with a flower on the heel (1)
a bottle of ginski vodka on a white surface (1)
two people holding cups of coffee with the word dunkin donuts on them (1)
a pair of sn

In [ ]:
#@markdown ### 📂 Unzip dataset
#@markdown It's much slower to upload individual files to your Drive, so you may want to upload a zip if you have your dataset in your computer.
zip = "/content/drive/MyDrive/Loras/example.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/example/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")


In [ ]:
#@markdown ### 🔢 Count datasets
#@markdown Google Drive makes it impossible to count the files in a folder, so this will show you the file counts in all folders and subfolders.
folder = "/content/drive/MyDrive/Loras" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
  if tree[path] and others:
    tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))


In [ ]:
if "step1_installed_flag" not in globals():
  raise Exception("Please run step 1 first!")

#@markdown ### 🚮 Clean folder
#@markdown Careful! Deletes all non-image files in the project folder.

!find {images_folder} -type f ! \( -name '*.png' -o -name '*.jpg' -o -name '*.jpeg' \) -delete
